__importing basic packages__

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mt

In [2]:
import tensorflow as tf
print(tf.__version__)
import keras

2.5.0


In [4]:
import cv2, os

In [5]:
# Preprocessing could include resize, nomalization, augmentation, changing color space, smoothening the images, morphological transfer, dilation & erosion

#In this case using - ImageDataGenerator -- for data augmentation

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Conv2D, Input, ZeroPadding2D
from keras.layers import BatchNormalization, Activation, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import RMSprop
from keras.models import Model, load_model
from keras.metrics import Accuracy
from keras import utils
from keras.models import Sequential

In [6]:
# import imutils

In [7]:
# defining various augmentation properties
train_datagen = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# dividing the total number of models into batches
batch_size=10

# Reading the train directory images
train_data = train_datagen.flow_from_directory('C:/Users/comp/Python Coding/LU AI-ML/Project 10 -- Mask Detection using open CV/data/train', target_size=(100,100), batch_size=batch_size, class_mode='binary')

Found 1376 images belonging to 2 classes.


In [9]:
# Reading the test directory images

test_data = test_datagen.flow_from_directory('C:/Users/comp/Python Coding/LU AI-ML/Project 10 -- Mask Detection using open CV/data/test', target_size=(100,100), batch_size=batch_size, class_mode='binary')

Found 194 images belonging to 2 classes.


In [10]:
# CNN model architechture

model = Sequential(
    [Conv2D(100,(3,3), activation='relu', input_shape = (100,100,3)), MaxPooling2D(2,2),
     Conv2D(100,(3,3), activation='relu'), MaxPooling2D(2,2),
     Flatten(),
     Dropout(0.5),
     Dense(50, activation='relu'),
     Dense(2, activation='softmax')])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 100)       2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 52900)             0         
_________________________________________________________________
dropout (Dropout)            (None, 52900)             0         
_________________________________________________________________
dense (Dense)                (None, 50)                2

In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
# using sparse_categorical_crossentropy instead of binary_crossentropy

In [13]:
# Train the model

history = model.fit_generator(train_data, steps_per_epoch=50//batch_size, epochs=10, validation_data=test_data, validation_steps=40//batch_size)

C:\Users\comp\anaconda3\lib\site-packages\keras\engine\training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
5/5 [==============================] - 60s 2s/step - loss: 1.4017 - acc: 0.4594 - val_loss: 0.7381 - val_acc: 0.4500
Epoch 2/10
5/5 [==============================] - 8s 2s/step - loss: 0.6897 - acc: 0.5061 - val_loss: 0.6956 - val_acc: 0.4500
Epoch 3/10
5/5 [==============================] - 8s 2s/step - loss: 0.6927 - acc: 0.4781 - val_loss: 0.6438 - val_acc: 0.6500
Epoch 4/10
5/5 [==============================] - 7s 1s/step - loss: 0.7001 - acc: 0.4369 - val_loss: 0.6667 - val_acc: 0.5500
Epoch 5/10
5/5 [==============================] - 7s 1s/step - loss: 0.6714 - acc: 0.5789 - val_loss: 0.6271 - val_acc: 0.6500
Epoch 6/10
5/5 [==============================] - 7s 1s/step - loss: 0.6720 - acc: 0.5789 - val_loss: 0.6796 - val_acc: 0.4500
Epoch 7/10
5/5 [==============================] - 8s 2s/step - loss: 0.6586 - acc: 0.5467 - val_loss: 0.6725 - val_acc: 0.4500
Epoch 8/10
5/5 [==============================] - 8s 2s/step - loss: 0.6734 - acc: 0.4194 - val_loss: 0.6129 -

In [14]:
# Save the model
model.save('face_mask.h5')

In [15]:
model = tf.keras.models.load_model('face_mask.h5')

result = {0:'wihtout mask', 1:'with mask'}

In [16]:
gr_dict = {0:(0,0,255), 1:(0,255,0)}

In [17]:
rect_size = 4

In [18]:
cap = cv2.VideoCapture(0)

In [19]:
haarcascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
while True:
    r,im = cap.read()
    im = cv2.flip(im,1,1)
    # resize
    r_size = cv2.resize(im, (im.shape[1]//rect_size, im.shape[0]//rect_size))
    face = haarcascade.detectMultiScale(r_size)
    
    for i in face:
        (x,y,w,h) = [v * rect_size for v in i]
        face_img = im[y:y+h, x:x+w]
        r_size = cv2.resize(face_img, (100,100))
        normalize = r_size/255.0
        reshaped_img = np.reshape(normalize, (1,100,100,3))
        reshaped_img = np.vstack([reshaped_img])
        result = model.predict(reshaped_img)
        label = np.argmax(result, axis=1)[0]
        cv2.rectangle(im, (x,y), (x+w, y+h), gr_dict[label], 2)
        cv2.rectangle(im, (x, y-40), (x+w, y), gr_dict[label], -1)
        #cv2.putText(im, result[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    cv2.imshow('Recording', im)
    key=cv2.waitKey(10)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()